This is to receive and transmit button presses 
(or wifi connections from upstairs) and display on screen
timeouts for the heating system at home.
(Possibly measure temperatures later on)

In [44]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [ ]:
# Create the files in the disk that we need
%sendtofile --quiet --source ../basicsockets/syslog.py
%sendtofile --quiet --source ../essential_sensor_code/asyncdevices.py
%sendtofile --quiet --source ../basicsockets/connecthotspot.py
%sendtofile --quiet --source ../async_web_serve/webserve_funcs.py


In [45]:
%sendtofile main.py

import machine

#
# Make the screen code
#
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
import framebuf
buffer = bytearray(((64 // 8) * 128) + 1)
buffer[0] = 0x40  # Set first byte of data buffer to Co=0, D/C=1
fbuff = framebuf.FrameBuffer1(memoryview(buffer)[1:], 128, 64)
for c in b'\xae \x00@\xa1\xa8?\xc8\xd3\x00\xda\x12\xd5\x80\xd9\xf1\xdb0\x81\xff\xa4\xa6\x8d\x14\xaf':
    i2c.writeto(0x3c, bytes((0x80, c)))
def oledshow():
    i2c.writeto(0x3c, b'\x80!\x80\x00\x80\x7f\x80"\x80\x00\x80\x07')
    i2c.writeto(0x3c, buffer)

for i in range(0, 128, 8):
    for j in range(0, 64, 8):
        fbuff.fill_rect(i, j, 8, 8, (i//8 + j//8)%2)
oledshow()

Sent 20 lines (665 bytes) to main.py.


In [46]:
%sendtofile --append main.py

import time
time.sleep(1.0)

import os, ustruct, uasyncio
uasyncio.set_debug(0)

from connecthotspot import ipnumber
from webserve_funcs import servestaticfile
from webserve_funcs import servefilelist
from webserve_funcs import readhttpheaders
from webserve_funcs import servemessage


Sent 11 lines (284 bytes) to main.py.


In [47]:
%sendtofile --append main.py

tsecs_ontill = time.time()
n = 0
async def handleconnection(reader, writer):
    global n, tsecs_ontill
    n += 1
    receivedrequest = await readhttpheaders(reader)
    print(receivedrequest)

    rpath = receivedrequest.get("GET")
    fbuff.fill(0)
    fbuff.text("http", 0,0,1)
    fbuff.text(str(n), 80,0,1)
    fbuff.text(str(rpath), 0,10,1)
    oledshow()
    
    if rpath == "":
        await servefilelist(writer)
    elif rpath[:3] == "on?":
        await servemessage(writer, str(tsecs_ontill-time.time()))
    elif rpath[:3] == "on+":
        tsecs_ontill = time.time()+int(rpath[3:])
        await servemessage(writer, "tsecs_ontill %d"%tsecs_ontill)
    elif rpath:
        await servestaticfile(writer, rpath)
        
    await writer.aclose()


Sent 26 lines (761 bytes) to main.py.


In [48]:
%sendtofile --append main.py

pcontrol = machine.Pin(13, machine.Pin.OUT)
async def handletiming():
    while True:
        t = time.time()
        pcontrol.value(t<tsecs_ontill)
        if t%10 == 0:
            fbuff.fill_rect(0,32,128,32,0)
            fbuff.text(str(tsecs_ontill-t), 0, 38, 1)
            oledshow()
        await uasyncio.sleep(1)
    

Sent 11 lines (327 bytes) to main.py.


In [49]:
%sendtofile --append main.py

# enables rewriting handleconnection function without restarting
def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))
    except TypeError as e:
        print("handleconnection exception", str(e))

port = 80
loop = uasyncio.get_event_loop()
print("* Running on http://%s:%s/" % (ipnumber, port))
loop.create_task(uasyncio.start_server(handleconnection_indirect, ipnumber, port))
loop.create_task(handletiming())
    
loop.run_forever()


Sent 16 lines (564 bytes) to main.py.
